#### [ TEST ]


[1] 모듈 로딩 및 데이터 준비<hr>

In [1]:
## [1-1] 모듈 로딩
import cv2                           # 컴퓨터비젼 모듈
import numpy as np                   # 이미지 데이터 저장 모듈
import matplotlib.pyplot as plt      # 시각화 모듈
import os                            # 파일, 폴더, 경로 관련 모듈

In [2]:
## [1-2] 데이터 준비
FILE_CSV = '../data/cat_dog.csv'

In [3]:
if os.path.exists(FILE_CSV):
    print(f'{FILE_CSV} Dataset 파일이 없습니다.')

[2] 이미지 데이터 로딩 <hr>

In [4]:
## [2-1] 피쳐와 타겟 분리
import pandas as pd 

catdogDF=pd.read_csv(FILE_CSV, header=None)
catdogDF.info()

FileNotFoundError: [Errno 2] No such file or directory: '../data/cat_dog.csv'

[3] 학습 준비<hr>
- 피쳐와 타겟 분리
- 피쳐 스케일링
- 학습용, 테스트용 데이터셋 분리

In [ ]:
## [3-1] 피쳐 & 타겟 분리
featureDF = catdogDF[catdogDF.columns[1:]]
targetSR = catdogDF[catdogDF.columns[0]]

print(f'featureDF : {featureDF.shape}   targetSR : {targetSR.shape}')

In [ ]:
## [3-2] 피쳐 정규화
featureDF = featureDF/255.
featureDF.describe()

In [ ]:
## [3-3] 학습용, 테스트용 데이터셋 분리
from sklearn.model_selection import train_test_split

x_train ,x_test, y_train, y_test = train_test_split(featureDF, targetSR,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=targetSR)

In [ ]:
print(f'x_train : {x_train.shape}   y_train : {y_train.shape}')
print(f'x_test : {x_test.shape}   y_test : {y_test.shape}')

[4] 학습 진행<hr>
- 최적 모델 체크
- 기준선 설정 
- 학습 진행

In [ ]:
## [4-1] 최적 모델 체크
from sklearn.utils.discovery import *
from sklearn.metrics import *
import warnings

# {“classifier”, “regressor”, “cluster”, “transformer”} 
rets=all_estimators(type_filter='classifier')

In [ ]:
resultList=[]
for name, estimator_ in rets:
    try:
        model=estimator_()
        if 'Logistic' in name or 'SGD' in name or 'MLP' in name:
            model.set_params(max_iter=10000)
        if 'SV' in name:
            model.set_params(max_iter=100000, dual='auto')   
 
        model.fit(x_train, y_train)

        trainScore= model.score(x_train, y_train)
        testScore = model.score(x_test, y_test)

        resultList.append((name, round(trainScore, 3), round(testScore, 3)))
    except Exception:
        pass

In [ ]:
sorted(resultList, key=lambda x : x[2], reverse=True)

In [ ]:
## [4-1] 최적 모델 체크
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train , y_train)

In [ ]:
from sklearn.metrics import classification_report

trainScore= model.score(x_train, y_train)
testScore = model.score(x_test, y_test)
print('[TRAIN]' , trainScore , '[TEST]' , testScore)

pre = model.predict(x_test)
resultDict=classification_report(y_test, pre)
print(resultDict)

[5] 학습 및 튜닝 진행 <hr>